In [1]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

/Users/ojitha/GitHub/LLMTuning/sagemaker_examples/.venv/lib/python3.12/site-packages/sagemaker_core/main/shapes.py:6645: SyntaxWarning: invalid escape sequence '\|'
  domain: The machine learning domain of the model and its components. Valid Values: COMPUTER_VISION \| NATURAL_LANGUAGE_PROCESSING \| MACHINE_LEARNING
/Users/ojitha/GitHub/LLMTuning/sagemaker_examples/.venv/lib/python3.12/site-packages/sagemaker_core/main/shapes.py:7450: SyntaxWarning: invalid escape sequence '\*'
  schedule_expression: A cron expression that describes details about the monitoring schedule. The supported cron expressions are:   If you want to set the job to start every hour, use the following:  Hourly: cron(0 \* ? \* \* \*)    If you want to start the job daily:  cron(0 [00-23] ? \* \* \*)    If you want to run the job one time, immediately, use the following keyword:  NOW    For example, the following are valid cron expressions:   Daily at noon UTC: cron(0 12 ? \* \* \*)    Daily at midnight UTC: cron(0 0

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ojitha/Library/Application Support/sagemaker/config.yaml


/Users/ojitha/GitHub/LLMTuning/sagemaker_examples/.venv/lib/python3.12/site-packages/smdebug_rulesconfig/actions/utils.py:5: SyntaxWarning: invalid escape sequence '\-'
  TRAINING_JOB_PREFIX_REGEX = "^[A-Za-z0-9\-]+$"
/Users/ojitha/GitHub/LLMTuning/sagemaker_examples/.venv/lib/python3.12/site-packages/smdebug_rulesconfig/actions/utils.py:6: SyntaxWarning: invalid escape sequence '\w'
  EMAIL_ADDRESS_REGEX = "^[a-z0-9]+[@]\w+[.]\w{2,3}$"
/Users/ojitha/GitHub/LLMTuning/sagemaker_examples/.venv/lib/python3.12/site-packages/smdebug_rulesconfig/actions/utils.py:7: SyntaxWarning: invalid escape sequence '\+'
  PHONE_NUMBER_REGEX = "^\+\d{1,15}$"


In [5]:
model_id, model_version = "huggingface-text2text-flan-t5-xl", "1.2.2"

Using SageMaker, we can perform inference on the pre-trained model, even without fine-tuning it first on a new dataset. We start by retrieving the `deploy_image_uri`, `deploy_source_uri`, and `model_uri` for the pre-trained model. To host the pre-trained model, we create an instance of [`sagemaker.model.Model`](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html) and deploy it. This may take a few minutes.

In [6]:
def get_sagemaker_session(local_download_dir) -> sagemaker.Session:
    """Return the SageMaker session."""

    sagemaker_client = boto3.client(
        service_name="sagemaker", region_name=boto3.Session().region_name
    )

    session_settings = sagemaker.session_settings.SessionSettings(
        local_download_dir=local_download_dir
    )

    # the unit test will ensure you do not commit this change
    session = sagemaker.session.Session(
        sagemaker_client=sagemaker_client, settings=session_settings
    )

    return session

This text-to-text generation task supports a wide variety of model sizes that have different compute requirements. Here, we specify the instance type for several large models along with an environment variable to set the multi-model endpoint number of workers to 1. This ensures we can support the largest possible token lengths since additional models are not consuming GPU memory resources.

In [7]:
## CODE CELL 6 ##
_large_model_env = {"SAGEMAKER_MODEL_SERVER_WORKERS": "1", "TS_DEFAULT_WORKERS_PER_MODEL": "1"}

_model_config_map = {
    "huggingface-text2text-flan-t5-xxl": {
        "instance_type": "ml.g5.12xlarge",
        "env": _large_model_env,
    },
    "huggingface-text2text-flan-t5-xxl-fp16": {
        "instance_type": "ml.g5.12xlarge",
        "env": _large_model_env,
    },
    "huggingface-text2text-t5-one-line-summary": {
        "instance_type": "ml.g5.2xlarge",
        "env": _large_model_env,
    },
    "huggingface-text2text-flan-t5-xl": {
        "instance_type": "ml.g5.2xlarge",
        "env": {"MMS_DEFAULT_WORKERS_PER_MODEL": "1"},
    },
    "huggingface-text2text-flan-t5-large": {
        "instance_type": "ml.g5.2xlarge",
        "env": {"MMS_DEFAULT_WORKERS_PER_MODEL": "1"},
    },
    "huggingface-text2text-flan-ul2-bf16": {
        "instance_type": "ml.g5.12xlarge",
        "env": _large_model_env,
    },
}

In [8]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base


endpoint_name = name_from_base(f"jumpstart-example-{model_id}")

if model_id in _model_config_map:
    inference_instance_type = _model_config_map[model_id]["instance_type"]
else:
    inference_instance_type = "ml.g5.2xlarge"

# Retrieve the inference docker container uri. This is the base HuggingFace container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

# Retrieve the inference script uri. This includes all dependencies and scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

# Retrieve the model uri.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)


In [12]:
model_uri

's3://jumpstart-cache-prod-ap-southeast-2/huggingface-infer/prepack/v1.1.2/infer-prepack-huggingface-text2text-flan-t5-xl.tar.gz'

In [13]:
# Create the SageMaker model instance
if model_id in _model_config_map:
    # For those large models, we already repack the inference script and model
    # artifacts for you, so the `source_dir` argument to Model is not required.
    model = Model(
        image_uri=deploy_image_uri,
        model_data=model_uri,
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
        env=_model_config_map[model_id]["env"],
    )
else:
    model = Model(
        image_uri=deploy_image_uri,
        source_dir=deploy_source_uri,
        model_data=model_uri,
        entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
        sagemaker_session=get_sagemaker_session("download_dir"),
    )

# Deploy the Model. Note that we need to pass Predictor class when we deploy model through the Model class
# defined above, so we can run inference through the sagemaker API.
model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
)

----------!

Input to the endpoint is any string of text formatted as json and encoded in `utf-8` format. Output of the endpoint is a `json` with generated text.

In [14]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"


def query_endpoint(encoded_text, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/x-text", Body=encoded_text
    )
    return response


def parse_response(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_text"]
    return generated_text

Below, we put in some example input text. You can put in any text and the model predicts next words in the sequence. Longer sequences of text can be generated by calling the model repeatedly.

In [15]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"

text1 = "Translate to German:  My name is Arthur"
text2 = "Recipe: How to make bolognese pasta"
text3 = "Summarize: We describe a system called Overton, whose main design goal is to support engineers in building, monitoring, and improving production \
machine learning systems. Key challenges engineers face are monitoring fine-grained quality, diagnosing errors in sophisticated applications, and \
handling contradictory or incomplete supervision data. Overton automates the life cycle of model construction, deployment, and monitoring by providing a \
set of novel high-level, declarative abstractions. Overtons vision is to shift developers to these higher-level tasks instead of lower-level machine learning tasks. \
In fact, using Overton, engineers can build deep-learning-based applications without writing any code in frameworks like TensorFlow. For over a year, \
Overton has been used in production to support multiple applications in both near-real-time applications and back-of-house processing. In that time, \
Overton-based applications have answered billions of queries in multiple languages and processed trillions of records reducing errors 1.7-2.9 times versus production systems."

f = open("diy_results.txt","w")
f.write(f"{endpoint_name}{newline}")

for text in [text1, text2, text3]:
    query_response = query_endpoint(text.encode("utf-8"), endpoint_name=endpoint_name)
    generated_text = parse_response(query_response)
    f.write(f"generated text: {generated_text}{newline}")
    print(
        f"Inference:{newline}"
        f"input text: {text}{newline}"
        f"generated text: {bold}{generated_text}{unbold}{newline}"
    )
    
f.close()    
    

Inference:
input text: Translate to German:  My name is Arthur
generated text: Ich bin Arthur.

Inference:
input text: Recipe: How to make bolognese pasta
generated text: Bring a large pot of water to a boil. Add the pasta and cook according to

Inference:
input text: Summarize: We describe a system called Overton, whose main design goal is to support engineers in building, monitoring, and improving production machine learning systems. Key challenges engineers face are monitoring fine-grained quality, diagnosing errors in sophisticated applications, and handling contradictory or incomplete supervision data. Overton automates the life cycle of model construction, deployment, and monitoring by providing a set of novel high-level, declarative abstractions. Overtons vision is to shift developers to these higher-level tasks instead of lower-level machine learning tasks. In fact, using Overton, engineers can build deep-learning-based applications without writing any code in frameworks like T

In [ ]:
by using the huggingface-text2text-t5-one-line-summary LLM.

## Cleanup AWS Resources

To avoid ongoing charges, it's important to clean up the AWS resources created during this notebook execution. This includes deleting the SageMaker endpoint and model.

In [16]:
# Delete the SageMaker endpoint
try:
    model_predictor.delete_endpoint(delete_endpoint_config=True)
    print(f"Successfully deleted endpoint: {endpoint_name}")
except Exception as e:
    print(f"Error deleting endpoint {endpoint_name}: {str(e)}")

# Delete the SageMaker model
try:
    model.delete_model()
    print(f"Successfully deleted model: {endpoint_name}")
except Exception as e:
    print(f"Error deleting model {endpoint_name}: {str(e)}")

print("Cleanup completed!")

Successfully deleted endpoint: jumpstart-example-huggingface-text2text-2025-09-20-03-30-36-375
Successfully deleted model: jumpstart-example-huggingface-text2text-2025-09-20-03-30-36-375
Cleanup completed!
